In [ ]:
import os
import math
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
import time
import copy
from datetime import datetime
import sys
import cv2
sys.path.append("..")
from training_loop import train_model_multi_class

In [ ]:
BATCH_SIZE = 20
LABELS = sorted([i[:-4] for i in os.listdir('icons')])
NUM_CLASSES = len(LABELS)

In [ ]:
import numpy as np
import collections

class GaussianNoise(object):
    """
    Add gaussian noise to a numpy.ndarray (H x W x C)
    """
    def __init__(self, mean, sigma, random_state=np.random):
        self.sigma = sigma
        self.mean = mean
        self.random_state = random_state

    def __call__(self, image):
        row, col, ch = image.shape
        gauss = self.random_state.normal(self.mean, self.sigma, (row, col, ch))
        gauss = gauss.reshape(row, col, ch)
        image += torch.from_numpy(gauss).float()
        return image

In [ ]:
def remove_background(image):
    image = np.array(image)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    l_h = 0
    u_h = 179
    l_s = 0
    u_s = 60
    l_v = 0
    u_v = 255

    lower_green = np.array([l_h, l_s, l_v])
    upper_green = np.array([u_h, u_s, u_v])

    mask = cv2.inRange(hsv, lower_green, upper_green)
    mask_inv = cv2.bitwise_not(mask)

    fg = cv2.bitwise_and(image, image, mask=mask_inv)
    return Image.fromarray(fg)

In [ ]:
class IconsDataset(Dataset):
    def __init__(self, directory, labels, transform=None):
        self.directory = directory
        self.files = os.listdir(directory)[:10000]
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        f = self.files[index]
        # image
        image = Image.open(os.path.join(self.directory, f))
        if self.transform:
            image = self.transform(image)
        
        # labels    
        label_strings = f[:-4].split('_')
        labels = []
        for l in LABELS:
            if l in label_strings:
                labels.append(1.0)
            else:
                labels.append(0.0)
        return image, torch.tensor(labels, dtype=torch.float)
    
    def __len__(self):
        return len(self.files)


def load_dataset():
    transform = transforms.Compose([
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.0),
        remove_background,
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    d = IconsDataset('images_224/', LABELS, transform=transform)
    size = len(d)
    train, validate = random_split(d, [int(size * 0.8), int(size * 0.2)])
    
    loader = DataLoader(train, batch_size=BATCH_SIZE)
    validation_loader = DataLoader(validate, batch_size=BATCH_SIZE)

    return loader, validation_loader

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(512, NUM_CLASSES)

#state_dict = torch.load('vl_0.05622.pth')
#model.load_state_dict(state_dict)

model = model.to(device)

loader, validation_loader = load_dataset()
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(params=model.parameters(), lr=0.0001)
train_model_multi_class(
    'resnet18_4_reduced_colorjitter_no_background',
    model,
    {'train': loader, 'val': validation_loader},
    criterion,
    optimizer,
    device,
    num_epochs=1000
)